In [60]:
import json

# Baca file JSON
with open('/content/data_train (3).json', 'r') as file:
    candidates = json.load(file)

In [61]:
jobreq = "deskripsi kerja posisi lowong partisipasi prose bangun aplikasi sedang bangun usaha nama usaha harap mampu kerja tim responsibility buat modifikasi program tanggung jawab kelola program"

In [62]:
!pip install -q googletrans==4.0.0-rc1
!pip install -q googletrans==4.0.0-rc1 langdetect

In [63]:
from googletrans import Translator
from langdetect import detect

In [64]:
def translate_text(text):
    translator = Translator()
    try:
        # Detect the language
        lang = detect(text)
        # Translate only if the text is in Indonesian
        if lang == 'id':
            translated = translator.translate(text, src='id', dest='en')
            return translated.text
        else:
            # Return the original text if it's not in Indonesian
            return text
    except Exception as e:
        # If detection or translation fails, return the original text
        return text

In [65]:
from datetime import datetime

In [66]:
def calculate_duration(start_date, end_date):
    if start_date.lower() in ['none', 'unknown']:
        return "Unknown duration"

    # Try parsing the start_date in different formats
    try:
        start_date_obj = datetime.strptime(start_date, "%b %Y")
    except ValueError:
        try:
            start_date_obj = datetime.strptime(start_date, "%Y")
        except ValueError:
            return "Unknown duration"

    if end_date.lower() == 'present':
        end_date_obj = datetime.now()
    elif end_date.lower() in ['none', 'unknown']:
        return "Unknown duration"
    else:
        # Try parsing the end_date in different formats
        try:
            end_date_obj = datetime.strptime(end_date, "%b %Y")
        except ValueError:
            try:
                end_date_obj = datetime.strptime(end_date, "%Y")
            except ValueError:
                return "Unknown duration"

    # Calculate the difference between end_date and start_date
    duration = end_date_obj - start_date_obj

    # Extract years and months from the duration
    years = duration.days // 365
    months = (duration.days % 365) // 30

    # Format the duration
    if years >= 1:
        if years == 1:
            return "+1 year"
        elif years == 2:
            return "+2 years"
        else:
            return f"+{years} years"
    else:
        return f"{months} months"

In [67]:
# Function to translate candidates' data
def translate_candidate_data(candidate):
    # Translate work experience descriptions
    for experience in candidate.get("work_experience", []):
        # Translate job title
        job_title = experience.get("job_title", "")
        if job_title:
            translated_title = translate_text("".join(job_title))
            experience["job_title"] = translated_title

        # Translate job description
        if experience.get("job_desc"):
            experience["job_desc"] = [translate_text(desc) for desc in experience["job_desc"]]

        # Calculate duration
        duration = calculate_duration(experience.get("start_date", ""), experience.get("end_date", ""))
        experience["duration"] = duration

    # Translate education title and description if they exist
    for education in candidate.get("education", []):
        if isinstance(education, dict):
            # Translate education title
            education_title = education.get("title", "")
            if education_title:
                translated_education_title = translate_text("".join(education_title))
                education["title"] = translated_education_title

            # Translate education description
            if education.get("description"):
                education["description"] = translate_text(education["description"])

    # Translate languages
    candidate["languages"] = [translate_text(language) for language in candidate.get("languages", [])]

    # Translate skills
    candidate["skills"] = [translate_text(skill) for skill in candidate.get("skills", [])]


In [68]:
for candidate in candidates:
    translate_candidate_data(candidate)

In [ ]:
# Output the modified JSON for candidates
print(json.dumps(candidates, indent=5))

In [70]:
def translate_job_requirements(jobreq):
    translated_jobreq = {}

    # Translate the entire job requirements string
    translated_jobreq["job_requirements"] = translate_text(jobreq)

    return translated_jobreq

In [71]:
translate_jobreq = translate_job_requirements(jobreq)

In [ ]:
# Output the modified JSON for job requirements
print(json.dumps(translate_jobreq, indent=4))

In [73]:
import tensorflow as tf
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [74]:
!pip -q install Sastrawi

In [75]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [76]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Initialize stopwords and lemmatizer
english_stop_words = set(stopwords.words('english'))
stopword_factory = StopWordRemoverFactory()
indonesian_stop_words = set(stopword_factory.get_stop_words())
all_stop_words = english_stop_words.union(indonesian_stop_words)
lemmatizer = WordNetLemmatizer()

In [77]:
def text_preprocessing(text):
    if text is None:
        return ''

    if isinstance(text, dict):
        text = preprocess_job_requirements(text)

    text_no_punct = ''.join([char for char in text if char not in string.punctuation])
    text_lower = text_no_punct.lower()
    text_single_line = ' '.join(text_lower.split())
    tokens = word_tokenize(text_single_line)
    tokens_cleaned = [lemmatizer.lemmatize(word) for word in tokens if word not in all_stop_words]
    clean_text = ' '.join(tokens_cleaned)

    return clean_text

In [78]:
def preprocess_job_requirements(jobreq):
    # Preprocess the entire job requirements string
    preprocessed_jobreq = text_preprocessing(jobreq)

    return {
        'job_requirements': preprocessed_jobreq
    }

In [79]:
clean_jobreq = preprocess_job_requirements(translate_jobreq["job_requirements"])

In [ ]:
clean_jobreq

In [81]:
def preprocess_candidate(candidate):
    work_experience = []
    for experience in candidate.get('work_experience', []):
        preprocessed_experience = {
            'job_title': text_preprocessing(experience.get('job_title', '')),
            'duration': text_preprocessing(experience.get('duration', ''))
        }
        work_experience.append(preprocessed_experience)

    education = []
    for edu in candidate.get('education', []):
        if isinstance(edu, dict):
            preprocessed_edu = {
                'title': text_preprocessing(edu.get('title', '')),
            }
            education.append(preprocessed_edu)

    languages = [text_preprocessing(language) if language != 'Indonesia' else language for language in candidate.get('languages', [])]
    skills = [text_preprocessing(skill) for skill in candidate.get('skills', [])]

    certification = []
    for cert in candidate.get('certification', []):
        if isinstance(cert, dict):
            preprocessed_cert = {
                'title': text_preprocessing(cert.get('title', '')),
            }
            certification.append(preprocessed_cert)

    return {
        'basic_info': candidate.get('basic_info', {}),
        'work_experience': work_experience,
        'education': education,
        'languages': languages,
        'skills': skills,
        'certification': certification
    }

In [82]:
def preprocess_candidates(candidates):
    preprocessed_candidates = []
    for candidate in candidates:
        preprocessed_candidate = preprocess_candidate(candidate)
        preprocessed_candidates.append(preprocessed_candidate)
    return preprocessed_candidates


In [83]:
clean_candidates = preprocess_candidates(candidates)

In [ ]:
print(json.dumps(clean_candidates, indent=4))

In [103]:
with open("data.json", "w") as final:
    json.dump(clean_candidates, final)

In [ ]:
f = open('data.json')

# returns JSON object as
# a dictionary
clean_candidates = json.load(f)

In [101]:
# Initialize stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to preprocess text
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords and lemmatize
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words)
    return text

# Function to combine and preprocess candidate data
def preprocess_candidate_data(candidate):
    combined_text = []
    # Combine basic info
    basic_info = candidate['basic_info']
    combined_text.append(basic_info['name'])
    combined_text.append(basic_info['location'])

    # Combine work experience
    for work in candidate['work_experience']:
        for i in range (10):
          combined_text.append(work['job_title'])
        if 'company' in work.keys():
          combined_text.append(work['company'])
        combined_text.append(work['location'])
        combined_text.append(work['start_date'])
        combined_text.append(work['end_date'])
        if 'job_desc' in work.keys():
          combined_text.extend(work['job_desc'])

    # Combine education
    for education in candidate['education']:
        combined_text.append(education['title'])


    # Combine languages, skills, and certifications
    combined_text.extend(candidate['languages'])
    combined_text.extend(candidate['skills'])
    for certification in candidate['certification']:
        for i in range (10):
          combined_text.append(certification['title'])
        combined_text.append(certification['issuer'])

    # Join all text into a single string
    combined_text = ' '.join(combined_text)
    # Preprocess the combined text
    return preprocess_text(combined_text)

# Preprocess job requirements
preprocessed_jobreq = preprocess_text(jobreq)

In [ ]:
# Translate and preprocess job requirement
translated_jobreq = translate_text(jobreq)
preprocessed_jobreq = preprocess_text(translated_jobreq)
print(preprocessed_jobreq)

In [ ]:
# Translate and preprocess all candidates
preprocessed_candidates = [preprocess_candidate_data(candidate) for candidate in candidates]

# Print the preprocessed candidates
for i, preprocessed_candidate in enumerate(preprocessed_candidates):
    print(f"Candidate {i+1}:")
    print(preprocessed_candidate)
    print()

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Create a dummy jobreq_data DataFrame
jobreq_data = pd.DataFrame({
    'description': [preprocessed_jobreq]
})

# Assuming preprocessed_candidates is a list of preprocessed candidate strings
cv_data = pd.DataFrame({
    'Text': preprocessed_candidates
})

In [ ]:
jobreq_data

In [ ]:
cv_data

In [ ]:
cv_data.to_csv('data_train.csv')

In [ ]:
# Load the saved model
model = load_model('/content/collaborative_filtering_model (2).h5')

In [ ]:
# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the CV data
vectorizer.fit(cv_data['Text'])

# Transform the job requirement description
jobreq_tfidf = vectorizer.transform(jobreq_data['description'])

# Define a function to recommend CVs for a job requirement using the trained collaborative filtering model
def recommend_cvs_for_job(jobreq_id, model, vectorizer, jobreq_data, cv_data, top_n=5):
    # Transform the job description to its TF-IDF representation
    jobreq_tfidf = vectorizer.transform([jobreq_data.loc[jobreq_id, 'description']])
    # Get the indices of CVs
    cv_indices = np.arange(len(cv_data))
    # Repeat the job requirement index for all CVs
    job_indices = np.repeat(jobreq_id, len(cv_data))
    # Predict the points using the model
    predicted_points = model.predict([cv_indices, job_indices])
    # Get the top recommended CVs
    top_cvs_indices = predicted_points.squeeze().argsort()[::-1][:top_n]
    # Get the CV texts and similarity scores
    top_cvs_with_scores = []
    for cv_idx in top_cvs_indices:
        cv_text = cv_data.loc[cv_idx, 'Text']
        similarity_score = predicted_points[cv_idx]
        top_cvs_with_scores.append((cv_text, similarity_score))
    return top_cvs_with_scores

# Example: get top 5 CVs for a specific job requirement using the collaborative filtering model
jobreq_id = 0  # Replace with a job requirement index for testing

top_cvs_with_scores = recommend_cvs_for_job(jobreq_id, model, vectorizer, jobreq_data, cv_data)

print("Top CVs for Job Requirement ID {}: ".format(jobreq_id))
for cv_text, similarity_score in top_cvs_with_scores:
    print(f"CV: {cv_text}\nSimilarity Score: {similarity_score}\n")

# SAMPAI SINI AJA DULU YG BENER

In [ ]:
cv_data.to_csv('data_train.csv')